<a href="https://colab.research.google.com/github/Dembelinho/Langchain_Agents/blob/main/Langchain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [24]:
!pip install --upgrade --quiet langchain_text_splitters

In [25]:
!pip install --upgrade --quiet beautifulsoup4

In [26]:
!pip install --upgrade --quiet langchain

In [27]:
!pip install --upgrade --quiet cohere

In [28]:
!pip install --upgrade --quiet langchain-cohere

In [29]:
!pip install --upgrade --quiet faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.3 MB/s eta 0:00:00


In [37]:
%pip install --upgrade --quiet  docx2txt

  Preparing metadata (setup.py) ... done


In [47]:
%pip install --upgrade --quiet  langchain sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00


In [57]:
%pip install --upgrade --quiet langchain-fireworks

# Importation

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import CohereEmbeddings
from langchain_core.documents import Document
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [44]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere

# Define Tools

We will use two tools: Tavily (to search online) and then a retriever over a local index we will create

**Tavily’s Search API** is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

In [70]:
from google.colab import userdata

%env TAVILY_API_KEY=userdata.get('TAVILY_API_KEY') #put ur ravily api key

env: TAVILY_API_KEY="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" #put ur ravily api key


In [33]:
search = TavilySearchResults()

In [73]:
loader = WebBaseLoader("https://docs.cohere.com/docs/the-cohere-platform")

docs = loader.load()

Documents Loading

In [71]:
from langchain_community.document_loaders import Docx2txtLoader
loader1 = Docx2txtLoader("/content/drive/MyDrive/thingsboars.docx") #path of ur docx
docs1 = loader1.load()

In [75]:
from google.colab import userdata
#Embedding by cohere
embeddings = CohereEmbeddings(cohere_api_key=userdata.get('COHERE_API_KEY')) # put ur cohere api key

In [74]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [50]:
# Embeddings by HiggingFace
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [76]:
vector = FAISS.from_documents(documents, embeddings)

In [52]:
import os
os.environ["FIREWORKS_API_KEY"] =userdata.get('FIREWORKS_API_KEY')

In [72]:
cohere_llm = ChatCohere(cohere_api_key=userdata.get('COHERE_API_KEY'))

In [59]:
from langchain_fireworks import ChatFireworks
mistral_llm = ChatFireworks(model="accounts/fireworks/models/mistral-7b-instruct-4k")

In [62]:
prompt = ChatPromptTemplate.from_template("""Répondez à la question suivante en vous basant uniquement sur le contexte fourni :

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(mistral_llm, prompt)

In [68]:
document_chain.invoke({
    "input": "La page d'accueil est divisée en plusieurs sections lesquels?",
    "context": [Document(page_content="page d'accueil")]
})

"La page d'accueil est divisée en plusieurs sections qui peuvent inclure une section d'introduction, une section sur les produits ou services offerts, une section sur l'entreprise, une section sur les contacts et une section sur les liens vers les autres pages de l'entreprise."

A **retriever** is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them.

Retrievers accept a string query as input and return a list of Document's as output.

In [77]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [78]:
response = retrieval_chain.invoke({"input": "Cohere Large Language Models ?"})
print(response["answer"])


Cohere Large Language Models (LLMs) are a set of pre-trained models that can be used to generate text, perform text classification, and perform text-based tasks such as summarization and chatbots. The Command family of models includes Command, Command R, and Command R+, which are used for text generation and conversational agents. The Rerank model is used to inject the intelligence of a language model into an existing search system. The Embed model improves the accuracy of search, classification, clustering, and RAG results. The Cohere Platform serves as a foundation for building LLM-powered applications, and the Chat endpoint can be used to build a conversational agent powered by the Command family of models. Retrieval-Augmented Generation (RAG) is a feature that allows an LLM to access external data sources, leading to better, more factual generations. Fine-tuning can be done with generative models


In [79]:
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(cohere_llm, retriever, prompt)

In [80]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can i use Cohere for embedding?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='The Cohere Platform', metadata={'source': 'https://docs.cohere.com/docs/the-cohere-platform', 'title': 'The Cohere Platform', 'description': 'Cohere offers world-class Large Language Models (LLMs) like Command, Rerank, and Embed. These help developers and enterprises build LLM-powered applications such as conversational agents, summarization, and search systems. They also provide fine-tuning options and private, secure deployment choices.', 'language': 'en'}),
 Document(page_content="Click here to learn more about Cohere foundation models.\nThese LLMs Make it Easy to Build Conversational Agents (and Other LLM-powered Apps)\nTry Coral to see what an LLM-powered conversational agent can look like. It is able to converse, summarize text, and write emails and articles.\n\nOur goal, however, is to enable you to build your own LLM-powered applications. The Chat endpoint, for example, can be used to build a conversational agent powered by the Command family of models. 

In [81]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(mistral_llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [82]:
chat_history = [HumanMessage(content="Can i use Cohere for embeddings?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can i use Cohere for embeddings?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='The Cohere Platform', metadata={'source': 'https://docs.cohere.com/docs/the-cohere-platform', 'title': 'The Cohere Platform', 'description': 'Cohere offers world-class Large Language Models (LLMs) like Command, Rerank, and Embed. These help developers and enterprises build LLM-powered applications such as conversational agents, summarization, and search systems. They also provide fine-tuning options and private, secure deployment choices.', 'language': 'en'}),
  Document(page_content="Click here to learn more about Cohere foundation models.\nThese LLMs Make it Easy to Build Conversational Agents (and Other LLM-powered Apps)\nTry Coral to see what an LLM-powered conversational agent can look like. It is able to converse, summarize text, and write emails and articles.\n\nOur goal, however, is to enable you to build your own

# Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1- The retriever we just created. This will let it easily answer questions about Cohere

2- A search tool. This will let it easily answer questions that require up-to-date information.


In [83]:
# Let's set up a tool for the retriever we just created
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "cohere_search",
    "Search for information about Cohere. For any questions about Cohere, you must use this tool!",
)

In [84]:
#We can now create a list of the tools we want to work with

tools = [retriever_tool, search]

------------ Learning and testing parts with tools made by user----------

In [85]:
%pip install --upgrade --quiet langchain

In [ ]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Create a tool
we need to create a tool to call. For this example, we will create a custom tool from a function.

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [ ]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [1]:
from google.colab import userdata

%env COHERE_API_KEY=userdata.get('COHERE_API_KEY')

env: COHERE_API_KEY=userdata.get('COHERE_API_KEY')


In [ ]:
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r")

In [ ]:
llm_with_tools = llm.bind_tools([multiply]) #We’ll use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool when appropriate

In [ ]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': None}]

## Invoking the tool

In [ ]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("Qu'est-ce que quatre fois 23")

92

## Agents

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
%pip install --upgrade --quiet langchain langchainhub

In [ ]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [ ]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [ ]:
tools = [multiply, add, exponentiate]

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

With an agent, we can ask questions that require arbitrarily-many uses of our tools

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three"
    }
)

--------install ollama-------------

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama start
!ollama run mistral

time=2024-04-17T13:34:20.686Z level=INFO source=images.go:817 msg="total blobs: 0"
time=2024-04-17T13:34:20.687Z level=INFO source=images.go:824 msg="total unused blobs removed: 0"
time=2024-04-17T13:34:20.687Z level=INFO source=routes.go:1143 msg="Listening on 127.0.0.1:11434 (version 0.1.32)"
time=2024-04-17T13:34:20.687Z level=INFO source=payload.go:28 msg="extracting embedded files" dir=/tmp/ollama1873135486/runners
time=2024-04-17T13:34:28.862Z level=INFO source=payload.go:41 msg="Dynamic LLM libraries [rocm_v60002 cpu cpu_avx cpu_avx2 cuda_v11]"
time=2024-04-17T13:34:28.862Z level=INFO source=gpu.go:121 msg="Detecting GPU type"
time=2024-04-17T13:34:28.862Z level=INFO source=gpu.go:268 msg="Searching for GPU management library libcudart.so*"
time=2024-04-17T13:34:28.876Z level=INFO source=gpu.go:314 msg="Discovered GPU libraries: [/tmp/ollama1873135486/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.2.140]"
time=2024-04-17T13:34:28.877Z level=INFO source=

Create agent using google vertex

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
pip install -qU langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 13.6 MB/s eta 0:00:00


In [ ]:
!pip install google-cloud-aiplatform
from google.cloud import aiplatform

# Initialize the Vertex AI SDK
aiplatform.init(project='MY-AGENTS-PROJECT')

In [ ]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro")

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)
# you should activate vertexIA

----------testing FIREWORKS IA-----------

In [ ]:
pip install -qU langchain-fireworks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

In [ ]:
os.environ["FIREWORKS_API_KEY"] =userdata.get('FIREWORKS_API_KEY')

In [ ]:
from langchain_fireworks import ChatFireworks

llm = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...
First, let's calculate 3 to the fifth power:

3^5 = 243

Next, let's find the sum of twelve and three:

12 + 3 = 15

Now, multiply the result by the fifth power of three:

243 * 15 = 3645

Finally, square the whole result:

3645^2 = 13,292,025

So, the result of taking 3 to the fifth power, multiplying that by the sum of twelve and three, and then squaring the whole result is 13,292,025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': "First, let's calculate 3 to the fifth power:\n\n3^5 = 243\n\nNext, let's find the sum of twelve and three:\n\n12 + 3 = 15\n\nNow, multiply the result by the fifth power of three:\n\n243 * 15 = 3645\n\nFinally, square the whole result:\n\n3645^2 = 13,292,025\n\nSo, the result of taking 3 to the fifth power, multiplying that by the sum of twelve and three, and then squaring the whole result is 13,292,025."}

In [ ]:
agent_executor.invoke(
    {
        "input": "Porter 3 à la cinquième puissance et multiplier le résultat par la somme de douze et de trois, puis élever le tout au carré."
    }
)



> Entering new AgentExecutor chain...
To solve this, we first calculate 3 to the 5th power:

3^5 = 243

Next, we find the sum of 12 and 3:

12 + 3 = 15

Then, we multiply the result by 243:

243 * 15 = 3645

Finally, we square the result:

3645^2 = 13,282,025

> Finished chain.


{'input': 'Porter 3 à la cinquième puissance et multiplier le résultat par la somme de douze et de trois, puis élever le tout au carré.',
 'output': 'To solve this, we first calculate 3 to the 5th power:\n\n3^5 = 243\n\nNext, we find the sum of 12 and 3:\n\n12 + 3 = 15\n\nThen, we multiply the result by 243:\n\n243 * 15 = 3645\n\nFinally, we square the result:\n\n3645^2 = 13,282,025'}